In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Multiply, Conv1D, MaxPooling1D, Flatten, Dense, ReLU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

# Load and preprocess the data
df = pd.read_csv('food_supplements.csv')
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')
df.set_index('Sold_date', inplace=True)
daily_data = df.pivot_table(values='Sold_quantity', index='Sold_date', columns='Product_details', aggfunc='sum').fillna(0)

# Scale the data
scaler = StandardScaler()
daily_data_scaled = scaler.fit_transform(daily_data)
daily_data_scaled = pd.DataFrame(daily_data_scaled, index=daily_data.index, columns=daily_data.columns)

# Prepare sequences
sequence_length = 14
prediction_length = 7
num_features = daily_data_scaled.shape[1]

def create_sequences(data, sequence_length, prediction_length):
    x, y = [], []
    for i in range(len(data) - sequence_length - prediction_length + 1):
        x.append(data.iloc[i:(i + sequence_length)].values)
        y.append(data.iloc[i + sequence_length:(i + sequence_length + prediction_length)].values)
    return np.array(x), np.array(y)

x_lstm, y_lstm = create_sequences(daily_data_scaled, sequence_length, prediction_length)

# Define the model building function
def build_model(num_features):
    input_layer = Input(shape=(sequence_length, num_features))
    lstm1 = LSTM(20, return_sequences=True)(input_layer)
    lstm2 = LSTM(20, return_sequences=True)(input_layer)
    combined_features = Multiply()([lstm1, lstm2])

    conv1 = Conv1D(64, 3, activation='relu')(combined_features)
    max_pool = MaxPooling1D(2)(conv1)
    conv2 = Conv1D(64, 3, activation='relu')(max_pool)
    flat = Flatten()(conv2)

    num_output_units = prediction_length * num_features
    dense = Dense(num_output_units)(flat)
    output_layer = Reshape((prediction_length, num_features))(dense)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

model = build_model(num_features)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[RootMeanSquaredError()])

# Model training
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(x_lstm, y_lstm, validation_split=0.2, epochs=100, batch_size=64, callbacks=[early_stopping], verbose=1)

# Output training results
print("Model training completed.")
print("Training Loss History:", history.history['loss'])
print("Validation Root Mean Squared Error:", history.history['val_root_mean_squared_error'])


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.9196 - root_mean_squared_error: 0.9588 - val_loss: 1.5229 - val_root_mean_squared_error: 1.2340
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8861 - root_mean_squared_error: 0.9413 - val_loss: 1.5493 - val_root_mean_squared_error: 1.2447
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8793 - root_mean_squared_error: 0.9376 - val_loss: 1.5354 - val_root_mean_squared_error: 1.2391
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8516 - root_mean_squared_error: 0.9228 - val_loss: 1.5357 - val_root_mean_squared_error: 1.2392
Model training completed.
Training Loss History: [0.8837652206420898, 0.8801397085189819, 0.8713476061820984, 0.863332211971283]
Validation Root Mean Squared Error: [1.234045386314392, 1.244708776473999, 1.2391114234924316, 1.2392209768295288]
